In [6]:
%pip install --upgrade xlrd 

Defaulting to user installation because normal site-packages is not writeable
  Using cached xlrd-2.0.1-py2.py3-none-any.whl.metadata (3.4 kB)
Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)
  Attempting uninstall: xlrd
    Found existing installation: xlrd 0.7.1
    Uninstalling xlrd-0.7.1:
      Successfully uninstalled xlrd-0.7.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
camelot 12.6.29 requires SQLAlchemy<0.8.0,>=0.7.7, but you have sqlalchemy 2.0.36 which is incompatible.
camelot 12.6.29 requires xlrd==0.7.1, but you have xlrd 2.0.1 which is incompatible.


In [58]:
#open pickle file to dataframe
import pandas as pd
import pickle
# with open('../datafiles/filtered_data.pkl', 'rb') as f:
#     df = pickle.load(f)
# Read the Excel file into a DataFrame
df = pd.read_excel("../datafiles/GRADE Form 11-8-21 HEARTBRAND-.xlsx", sheet_name="Individual Grades ", engine='openpyxl')
# print(df_excel.head())  # Displays first few rows of the Excel DataFrame

# # Set options to display all rows and columns in the DataFrame
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

# # Display the DataFrame
# print(df_excel)
# # # Set options to display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [44]:
df.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Grade Date: Monday,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,,285,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127,Unnamed: 128,Unnamed: 129,Unnamed: 130,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143,Unnamed: 144,Unnamed: 145,Unnamed: 146,Unnamed: 147,Unnamed: 148,Unnamed: 149,Unnamed: 150,Unnamed: 151,Unnamed: 152,Unnamed: 153,Unnamed: 154,Unnamed: 155,Unnamed: 156,Unnamed: 157,Unnamed: 158,Unnamed: 159,Unnamed: 160,Unnamed: 161,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,Unnamed: 172,Unnamed: 173,Unnamed: 174,Unnamed: 175,Unnamed: 176,Unnamed: 177,Unnamed: 178,Unnamed: 179,Unnamed: 180,Unnamed: 181,Unnamed: 182,Unnamed: 183,Unnamed: 184,Unnamed: 185,Unnamed: 186,Unnamed: 187,Unnamed: 188,Unnamed: 189,Unnamed: 190,Unnamed: 191,Unnamed: 192,Unnamed: 193,Unnamed: 194,Unnamed: 195,Unnamed: 196,Unnamed: 197,Unnamed: 198,Unnamed: 199,Unnamed: 200,Unnamed: 201,Unnamed: 202,Unnamed: 203,Unnamed: 204,Unnamed: 205,Unnamed: 206,Unnamed: 207,Unnamed: 208,Unnamed: 209,Unnamed: 210,Unnamed: 211,Unnamed: 212,Unnamed: 213,Unnamed: 214,Unnamed: 215,Unnamed: 216,Unnamed: 217,Unnamed: 218,Unnamed: 219,Unnamed: 220,Unnamed: 221,Unnamed: 222,Unnamed: 223,Unnamed: 224,Unnamed: 225,Unnamed: 226,Unnamed: 227,Unnamed: 228,Unnamed: 229,Unnamed: 230,Unnamed: 231,Unnamed: 232,Unnamed: 233,Unnamed: 234,Unnamed: 235,Unnamed: 236,Unnamed: 237,Unnamed: 238,Unnamed: 239,
0,NaN,NaN,Bovina Feeders/ Sugarland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN

In [59]:
df = df.iloc[5:].reset_index(drop=True)
df = df.dropna(axis=1, how='all')
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True) 
df.head()

,NaN,Carcass #,TAGS,NaN,NaN,Sex,Color,Weight,Quality,Yield,NaN,Member ID,Member Name,NaN
0,1.0,998,C0271200036,8485,0036,Steer,Black,763,CH,1,BF,C0271,Artesian Cattle,NaN
1,2.0,974,C0271200083,8485,0083,Steer,Red,947,CH,3,BF,C0271,Artesian Cattle,NaN
2,3.0,976,C0271209281,8485,9281,Steer,Black,735,CH,3,BF,C0271,Artesian Cattle,NaN
3,4.0,977,C0271200044,8485,0044,Steer,Black MOT,858,CH,3,BF,C0271,Artesian Cattle,NaN
4,5.0,978,C0271200158,8485,0158,Steer,Black,992,CH,3,BF,C0271,Artesian Cattle,NaN


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Grade Date: Monday,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,,285,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,Carcass #,TAGS,NaN,NaN,Sex,Color,Weight,Quality,Yield,NaN,Member ID,Member Name
1,1.0,998,C0271200036,8485,0036,Steer,Black,763,CH,1,BF,C0271,Artesian Cattle
2,2.0,974,C0271200083,8485,0083,Steer,Red,947,CH,3,BF,C0271,Artesian Cattle
3,3.0,976,C0271209281,8485,9281,Steer,Black,735,CH,3,BF,C0271,Artesian Cattle
4,4.0,977,C0271200044,8485,0044,Steer,Black MOT,858,CH,3,BF,C0271,Artesian Cattle


,NaN,Carcass #,TAGS,NaN,NaN,Sex,Color,Weight,Quality,Yield,NaN,Member ID,Member Name
0,1.0,998,C0271200036,8485,0036,Steer,Black,763,CH,1,BF,C0271,Artesian Cattle
1,2.0,974,C0271200083,8485,0083,Steer,Red,947,CH,3,BF,C0271,Artesian Cattle
2,3.0,976,C0271209281,8485,9281,Steer,Black,735,CH,3,BF,C0271,Artesian Cattle
3,4.0,977,C0271200044,8485,0044,Steer,Black MOT,858,CH,3,BF,C0271,Artesian Cattle


In [61]:
with open('../datafiles/cattleMaxDf.pkl', 'rb') as f:
    filteredDf = pickle.load(f)

In [64]:
filteredDf.head()

,Status/Disposal,Ear Tag,Ear Tag Location,Brand,Brand Location,Tattoo,Tattoo Location,Other ID,Other ID Location,Name,Registration Number,Breeder,Breeder Member Number,Owner,Owner Member Number,Date of Birth,Birth Method,Sire,Dam,Status,Type or Sex,Percent Purebred,Breed,Breed 1,Breed 2,Breed 3,Breed Comp 1,Breed Comp 2,Breed Comp 3,Horns,Color Markings,EPDType,EPD Reporting Period,Calving Ease Direct EPD,Calving Ease Direct Acc,Birth Weight EPD,Birth Weight Acc,Weaning Weight EPD,Weaning Weight Acc,Yearling Weight EPD,Yearling Weight Acc,Milk EPD,Milk Acc,Total Maternal EPD,Calving Ease Maternal EPD,Calving Ease Maternal Acc,Scrotal Circumference EPD,Scrotal Circumference Acc,Ribeye Area EPD,Ribeye Area Acc,Marbling EPD,Marbling Acc,Fat Thickness EPD,Fat Thickness Acc,Carcass Weight EPD,Carcass Weight Acc,Percent Retail Cuts EPD,Percent Retail Cuts Acc,Intramuscular Fat EPD,Intramuscular Fat Acc,Birth Weight,Adjusted Birth Weight,Birth Contemp Group,Birth contemporaries,Birth Weight Ratio,Birth Height,Calving Ease,Birth Code,Nurse Code,Twin code,Dam Teat Score,Dam Udder Score,Dam Gestation Length,Weaning Date,Weaning Weight Date,Weaning Weight,Adjusted Weaning Weight,Weaning Contemp Group,Weaning Contemporaries,Weaning Weight Ratio,Weaning WDA,Weaning Height,Weaning Frame Score,Weaning Mgmt Code,Weaning Pasture Code,Weaning Sheath/Namvel,Yearling Date,Yearling Weight Date,Yearling Weight,Adjusted Yearling Weight,Yearling Contemp Group,Yearling Contemporaries,Yearling Weight Ratio,Yearling WDA,Yearling ADG,Yearling ADG Ratio,Yearling Height,Yearling Frame Score,Yearling Mgmt Code,Yearling Pasture Code,Yearling Sheath/Navel
0,0,NaN,NaN,C0271,RH,8075,LE,NaN,NaN,ARTESIAN CATTLE C0271188075,1A128345,ARTESIAN CATTLE & FARMING LLC,291.0,ARTESIAN CATTLE & FARMING LLC,291.0,3/5/2018,NS,AF31519,X93873,Active,C,NaN,NaN,AA,AN,XX,50.0,25.0,25.0,P,NaN,P,20240808.0,-0.5,0.10,0.4,0.08,47.0,0.06,83.0,0.01,24.0,0.04,46.9990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,6144-6271,40.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,NaN,NaN,C0271,RH,9071,NaN,NaN,NaN,ARTESIAN CATTLE 9071,1A151658,ARTESIAN CATTLE & FARMING LLC,291.0,ARTESIAN CATTLE & FARMING LLC,291.0,2/26/2019,NS,AF36497,X94219,Active,C,NaN,NaN,AA,CH,XX,50.0,25.0,25.0,P,NaN,P,20240808.0,-3.8,0.12,2.3,0.08,46.0,0.05,84.0,0.01,26.0,0.04,49.4180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,15786-6,22.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,NaN,NaN,C0271,RH,1027,NaN,NaN,NaN,ARTESIAN 1603Z OLIVIA 1027,1A217728,ARTESIAN CATTLE & FARMING LLC,291.0,ARTESIAN CATTLE & FARMING LLC,291.0,3/16/2021,NS,AF36497,X93901,Active,C,NaN,NaN,AA,AN,XX,50.0,25.0,25.0,P,NaN,P,20240808.0,-3.8,0.12,2.3,0.08,46.0,0.05,84.0,0.01,26.0,0.04,49.4180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,18216-3,66.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,NaN,NaN,C0271,RH,1077,NaN,NaN,NaN,ARTESIAN 2163A AVA 1077,1A217778,ARTESIAN CATTLE & FARMING LLC,291.0,ROCKY MILL CATTLE CO.,1668.0,2/24/2021,NS,AF42330,X209885,Reference,C,NaN,NaN,AA,AR,NaN,50.0,50.0,NaN,P,NaN,P,20240808.0,-2.6,0.13,2.1,0.08,42.0,0.05,77.0,0.01,27.0,0.04,48.2155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,NaN,NaN,C0271,RH,1080,NaN,NaN,NaN,ARTESIAN CATTLE 1080,1A217781,ARTESIAN CATTLE & FARMING LLC,291.0,ARTESIAN CATTLE & FARMING LLC,291.0,2/22/2021,NS,AF42330,X209905,Active,C,NaN,NaN,AA,AR,NaN,50.0,50.0,NaN,P,NaN,P,20240808.0,-5.1,0.11,2.1,0.08,42.0,0.05,77.0,0.01,2